## Sequential A/B Testing

A/B testing (also known as split testing) is the process of comparing two versions of an asset and measuring the difference in performance.

Involves conducting the test on 2 versions of a single variable at a time. It goes with the belief that not more than one factor should be varied at the same time.

**Case Overview**:

SmartAd is a mobile first advertiser agency. 

The company provides an additional service called Brand Impact Optimiser (BIO), a lightweight questionnaire, served with every campaign to determine the impact of the ad they design.

The task at hand is to design a reliable hypothesis testing algorithm for the BIO service and determine whether the recent advertising campaign resulted in a significant lift in brand awareness.

**Data**:

The BIO data for this project is a “Yes” and “No” response of online users to the following question:


`Q: Do you know the brand SmartAd?`

      Yes
      No

The data has the following columns:
 **auction_id**,  **experiment**, **date**, **hour**, **device_make**, **platform_os**, **browser**, **yes**, **no**.


## Table of Contents
1. [Libraries](#Libraries)
2. [Dataset](#Dataset)
3. [Sample conditional SPRT](#Sample-conditional-SPRT)

    3.1 [conditional SPRT function](#conditional-SPRT-function)

    3.2 [Boundaries and Plots](#Boundaries-and-Plots)

    3.3 [Data Transformation](#Data-Transformation)

    3.4 [Data Summary plot and print functions](#Data-Summary-plot-and-print-functions)

    3.5 [Testing](#Testing)

### 1. Libraries

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd
import numpy as np
import math
from scipy.stats import binom
from math import *


import seaborn as sns
import matplotlib.pyplot as plt

import json

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### 2. Dataset

In [2]:
# function to fetch data
def fetch_data(id, file_name):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile(file_name)

  data=pd.read_csv(file_name)
  return data

In [3]:
# fetch the data
data = fetch_data('1YSn01vvlHKQaAIBtwIXRNd-oTaTuDN09', 'ABAdRecall.csv')
data.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


### 3. Sample conditional SPRT

#### 3.1 conditional SPRT function

In [4]:
def ConditionalSPRT(x,y,t1,alpha=0.05,beta=0.10,stop=None):
        if t1<=1:
            printLog('warning',"Odd ratio should exceed 1.")
        if (alpha >0.5) | (beta >0.5):
            printLog('warning',"Unrealistic values of alpha or beta were passed."
                     +" You should have good reason to use large alpha & beta values")
        if stop!=None:
            stop=math.floor(n0)

        def comb(n, k):
            return factorial(n) // factorial(k) // factorial(n - k)
        
        def lchoose(b, j):
            a=[]
            if (type(j) is list) | (isinstance(j,np.ndarray)==True):
                if len(j)<2:
                    j=j[0]
            if (type(j) is list) | (isinstance(j,np.ndarray)==True):
                for k in j:
                    n=b
                    if (0 <= k) & (k<= n):
                        a.append(math.log(comb(n,k)))
                    else:
                        a.append(0)
            else:
                n=b
                k=j
                if (0 <= k) & (k<= n):
                    a.append(math.log(comb(n,k)))
                else:
                    a.append(0)

            return np.array(a)

        def g(x,r,n,t1,t0=1):
            return -math.log(h(x,r,n,t1))+math.log(h(x,r,n,t0))

        def h(x,r,n,t=1):
            return f(r,n,t,offset=ftermlog(x,r,n,t))

        def f(r,n,t,offset=0):
            upper=max(0,r-n)
            lower=min(n,r)
            rng=list(range(upper,lower+1))
            return np.sum(fterm(rng,r,n,t,offset))

        def fterm(j,r,n,t,offset=0):
            ftlog=ftermlog(j,r,n,t,offset)
            return np.array([math.exp(ex) for ex in ftlog])

        def ftermlog(j,r,n,t,offset=0):
            xx=r-j
            lch=lchoose(n,j)
            lchdiff=lchoose(n,xx)
            lg=np.array(j)*math.log(t)
            lgsum=lch+lchdiff
            lgsum2=lgsum+lg
            lgdiff=lgsum2-offset

            return lgdiff

        def logf(r,n,t,offset=0):

            z=f(r,n,t,offset)
            if z>0:
                return math.log(z)
            else:
                return np.nan

        def clowerUpper(r,n,t1c,t0=1,alpha=0.05,beta=0.10):
            offset=ftermlog(math.ceil(r/2),r,n,t1c)
            z=logf(r,n,t1c,logf(r,n,t0,offset)+offset)
            a=-math.log(alpha/(1-beta))
            b=math.log(beta/(1-alpha))
            lower=b
            upper=1+a
            return (np.array([lower,upper])+z)/math.log(t1c/t0)
            
        l=math.log(beta/(1-alpha))
        u=-math.log(alpha/(1-beta))
        sample_size=min(len(x),len(y))
        n=np.array(range(1,sample_size+1))

        if stop!=None:
            n=np.array([z for z in n if z<=stop])
        x1=np.cumsum(x[n-1])
        r=x1+np.cumsum(y[n-1])
        stats=np.array(list(map(g,x1, r, n, [t1]*len(x1)))) #recurcively calls g

        clu=list(map(clowerUpper,r,n,[t1]*len(r),[1]*len(r),[alpha]*len(r), [beta]*len(r)))
        limits=[]
        for v in clu:
            inArray=[]
            for vin in v:
                inArray.append(math.floor(vin))
            limits.append(np.array(inArray))
        limits=np.array(limits)

        k=np.where((stats>=u) | (stats<=l))
        cvalues=stats[k]
        if cvalues.shape[0]<1:
            k= np.nan
            outcome='Unable to conclude.Needs more sample.'
        else:
            k=np.min(k)
            if stats[k]>=u:
                outcome=f'Exposed group produced a statistically significant increase.'
            else:
                outcome='There is no statistically significant difference between two test groups'
        if (stop!=None) & (k==np.nan):
            c1=clowerUpper(r,stop,t1,alpha,beta)
            c1=math.floor(np.mean(c1)-0.5)
            if x1[n0]<=c1:
                truncate_decision='h0'
                outcome='Maximum Limit Decision. The aproximate decision point shows their is no statistically significant difference between two test groups'
            else:
                truncate_decision='h1'
                outcome=f'Maximum Limit Decision. The aproximate decision point shows exposed group produced a statistically significant increase.'
            truncated=stop
        else:
            truncate_decision='Non'
            truncated=np.nan
        return (outcome,n, k,l,u,truncated,truncate_decision,x1,r,stats,limits)

#### 3.2 Boundaries and Plots

In [14]:
class SequentialTest:
  def __init__(t1 = 2, alpha = 0.05, beta = 0.1, stop = None):
    '''
    initialise startup variables
    '''
    if t1<=1:
        printLog('warning',"Odd ratio should exceed 1.")
    if (alpha >0.5) | (beta >0.5):
        printLog('warning',"Unrealistic values of alpha or beta were passed."
                  +" You should have good reason to use large alpha & beta values")
    if stop!=None:
        stop=math.floor(n0)
  
  def computeBoundaries(self,alpha, beta):
    '''
    This function shoud compute boundaries 
    '''
    a=math.log(beta/(1-alpha))
    b=math.log((1 - beta)/alpha)
    return a, b

  def plotTest(self):
    '''
    showing the cumulative statistical test (e.g., log probability ratio) and the upper and lower limits.
    '''

  def plotBoundaries(self, exposed):
    '''cumulative sums of exposed successes, bounded by the critical limits.
    '''
    


In [38]:
# e_df = pd.DataFrame(exposed)
# a = e_df.cumsum()
# a.columns = ['value']
# sns.lineplot(x = a.index, y = a.value)
b

2.8903717578961645

#### 3.3  Data Transformation

In [16]:
def transform_data(df):
  '''
  segment data into exposed and control groups
  consider that SmartAd runs the experment hourly, group data into hours. 
      Hint: create new column to hold date+hour and use df.column.map(lambda x:  pd.Timestamp(x,tz=None).strftime('%Y-%m-%d:%H'))
  create two dataframes with bernouli series 1 for posetive(yes) and 0 for negative(no)
    Hint: Given engagement(sum of yes and no until current observation as an array) and success (yes count as an array), the method generates random binomial distribution
        #Example
          engagement = np.array([5, 3, 3])
          yes = np.array([2, 0, 3])       
        Output is "[1] 1 0 1 0 0 0 0 0 1 1 1", showing a binary array of 5+3+3 values
        of which 2 of the first 5 are ones, 0 of the next 3 are ones, and all 3 of
        the last 3 are ones where the position the ones is randomly distributed within each group.
  '''

  # split dataset to control and exposed groups
  exposed = df.loc[df.experiment == 'exposed']  #exposed set
  control = df.loc[df.experiment == 'control']  #control set

  #datehour
  exposed['dateHour'] = pd.to_datetime(exposed.date)
  exposed.dateHour += pd.to_timedelta(exposed.hour, unit='h')
  exposed.dateHour = exposed.dateHour.map(lambda x:  pd.Timestamp(x,tz=None).strftime('%Y-%m-%d:%H'))

  control['dateHour'] = pd.to_datetime(control.date)
  control.dateHour += pd.to_timedelta(control.hour, unit='h')
  control.dateHour = control.dateHour.map(lambda x:  pd.Timestamp(x,tz=None).strftime('%Y-%m-%d:%H'))

  # groupby datehour
  df_exposed = exposed.groupby('dateHour').agg({'auction_id':'count', 'device_make':'count', 'platform_os':'count', 'browser':'count', 'yes':'sum', 'no':'sum'})
  df_control = control.groupby('dateHour').agg({'auction_id':'count', 'device_make':'count', 'platform_os':'count', 'browser':'count', 'yes':'sum', 'no':'sum'})

  # engagement
  df_exposed['engagement'] = df_exposed['yes'] + df_exposed['no']
  df_control['engagement'] = df_control['yes'] + df_control['no']

  # success
  df_exposed['success'] = df_exposed['yes'] 
  df_control['success'] = df_control['yes'] 

  # p of success
  global p_e, p_c
  p_e = sum(df_exposed['success']) / sum(df_exposed['engagement'])
  p_c = sum(df_control['success']) / sum(df_control['engagement'])

  # engagement and success to arrays then  p
  engagement_e = df_exposed['engagement'].to_numpy()
  engagement_c = df_control['engagement'].to_numpy()


  # data generation
  e = np.random.choice([0, 1], size=((np.sum(engagement_e)),), p=[p_e, 1-p_e])
  c = np.random.choice([0, 1], size=((np.sum(engagement_c)),), p=[p_c, 1-p_c])

  return e,c


#### 3.4 Data Summary plot and print functions

In [12]:
def plotDataSummary(exposed, control):
  'This function plots cummulated success'
  fig, ax = plt.subplots(figsize=(10,8))
  kwargs = {'cumulative': True}
  sns.distplot(control.success, hist_kws=kwargs, kde_kws=kwargs, color = 'black')
  sns.distplot(exposed.success, hist_kws=kwargs, kde_kws=kwargs, color = 'green')
  plt.title('A histogram indicating cummulative distributions of success in the 2 groups black: control, green:exposed')
  plt.ylabel('frequency')
  plt.xlabel('cummulative success')

def pretyPrintTestResult(self, test):
  '''This function print final test result. Json format is recommended. For example
  {
    "name": "",
    "engagementCountControl": ,
    "engagementCountExposed": ,
    "positiveCountControl": ,
    "positiveCountExposed": ,
    "ControlSuccessProbability": ,
    "ExposedSuccessProbability": ,
    "basePositiveRate": ,
    "significanceSign": ".",
    "lift": ,
    "oddRatio": ,
    "exactSuccessOddRate":,
    "confidenceIntervalLevel": ,
    "alpha": ,
    "beta": ,
    "power": ,
    "criticalValue": ,
    "lower critical(a)": 
    "upper critical(b)": ,
    "TotalObservation": 
  }'''

#### 3.5 Testing

##### 3.5.1 Parameters

In [18]:
'statistical parameters for SPRT'
alpha = 0.05
beta = 0.1

'Compute statistical lower and upper decision points such as a and b'
st = SequentialTest()
a, b = st.computeBoundaries(alpha = alpha, beta = beta)

##data processing here
exposed,control=transform_data(data)

# odd ratio
odd_ratio=(p_e/(1-p_e))/(p_c/(1-p_c))

##### 3.5.2 Testing

In [19]:
test = ConditionalSPRT(x = exposed,y = control,t1 = odd_ratio, alpha=alpha,beta=alpha)

In [20]:
test[0]

'Unable to conclude.Needs more sample.'

##### 3.5.3 Plots

In [39]:
!pip install sprt
import sprt

In [ ]:
##plot data summary
# plotDataSummary(exposed,control)

# 'Print test result.'
# pretyPrintTestResult(resultObject)

In [40]:
# generate the requirements file
!pip freeze > requirements.txt